# Génération de texte automatique

#### Ce notebook contient le code pour générer des descriptions de biens de manière automatique.

In [19]:
! pip install -r requirements.txt

In [1]:
import os
import pandas as pd
import s3fs

import re

from bs4 import BeautifulSoup

import spacy
import cld3

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku 

In [22]:
#import shutil
#shutil.make_archive("checkpoints_dir", 'zip', "checkpoints_dir")
#shutil.unpack_archive("checkpoints_dir.zip", "checkpoints_dir", "zip")

In [5]:
#fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'https://minio.lab.sspcloud.fr'}) 
#fs.get('projet-funathon/diffusion/data/paris/2021-04-10/listings.csv.gz', 'listings.csv.gz')

In [6]:
listings_d = pd.read_csv('listings.csv.gz',compression='gzip')

In [7]:
pd.options.display.max_columns = 74
listings_d.head(3)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2577,https://www.airbnb.com/rooms/2577,20210410122610,2021-04-14,Loft for 4 by Canal Saint Martin,"100 m2 loft (1100 sq feet) with high ceiling, ...",NaN,https://a0.muscache.com/pictures/09da057c-0120...,2827,https://www.airbnb.com/users/show/2827,Karine,2008-09-09,"Casablanca, Grand Casablanca, Morocco","Arrivée à Casablanca il y a 4 ans, je suis amo...",within a day,50%,67%,f,https://a0.muscache.com/im/pictures/user/17fe6...,https://a0.muscache.com/im/pictures/user/17fe6...,République,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,NaN,Entrepôt,NaN,48.86957,2.36127,Entire loft,Entire home/apt,4,NaN,2 baths,2.0,2.0,"[""Essentials"", ""Kitchen"", ""Heating"", ""Host gre...",$125.00,3,1125,3.0,3.0,1125.0,1125.0,3.0,1125.0,NaN,t,0,0,12,286,2021-04-14,1,0,0,2019-01-02,2019-01-02,100.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,f,1,1,0,0,0.04
1,3109,https://www.airbnb.com/rooms/3109,20210410122610,2021-04-13,zen and calm,<b>The space</b><br />I bedroom appartment in ...,Good restaurants<br />very close the Montparna...,https://a0.muscache.com/pictures/baeae9e2-cd53...,3631,https://www.airbnb.com/users/show/3631,Anne,2008-10-14,"Paris, Île-de-France, France",NaN,within a day,80%,0%,f,https://a0.muscache.com/im/users/3631/profile_...,https://a0.muscache.com/im/users/3631/profile_...,Alésia,1.0,1.0,"['email', 'phone', 'facebook', 'reviews']",t,f,"Paris, Île-de-France, France",Observatoire,NaN,48.83191,2.31870,Entire apartment,Entire home/apt,2,NaN,1 bath,NaN,1.0,"[""Smoke alarm"", ""Dishes and silverware"", ""Hair...",$60.00,2,30,2.0,2.0,30.0,30.0,2.0,30.0,NaN,t,30,60,90,365,2021-04-13,9,0,0,2016-12-27,2019-10-24,100.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,f,1,1,0,0,0.17
2,5396,https://www.airbnb.com/rooms/5396,20210410122610,2021-04-16,Explore the heart of old Paris,"Cozy, well-appointed and graciously designed s...","You are within walking distance to the Louvre,...",https://a0.muscache.com/pictures/52413/f9bf76f...,7903,https://www.airbnb.com/users/show/7903,Borzou,2009-02-14,"Istanbul, İstanbul, Turkey",We are both journalists based in the Middle Ea...,within an hour,100%,92%,f,https://a0.muscache.com/im/users/7903/profile_...,https://a0.muscache.com/im/users/7903/profile_...,Saint-Paul - Ile Saint-Louis,1.0,1.0,"['email', 'phone', 'facebook', 'google', 'revi...",t,t,"Paris, Ile-de-France, France",Hôtel-de-Ville,NaN,48.85247,2.35835,Entire apartment,Entire home/apt,2,NaN,1 bath,NaN,1.0,"[""Essentials"", ""TV with standard cable"", ""Dish...",$47.00,1,1125,1.0,1.0,1125.0,1125.0,1.0,1125.0,NaN,t,18,47,58,58,2021-04-16,237,17,3,2009-06-30,2021-04-12,90.0,9.0,9.0,10.0,10.0,10.0,9.0,7510402838018,f,1,1,0,0,1.65


In [8]:
listings_d.description[1550]

"Up to 5/6 people, this apartment is ideal for a family. At the foot of the building you will find all necessary shops<br />(Monoprix, pharmacy, bakery, cheese, grocery etc. ..)<br /> many restaurants and a recognized market.<br />Many lines underground:<br />5 minutes walk from the Metro:12 and 6.<br />Less than 10 minutes walk from the Metro 10, 8  and 13)<br />20' from the Eiffel Tower and Les Invalides<br /><br /><b>The space</b><br />Ideally located in the heart of the 15th arrondissement of Paris, in the neighborhood Sèvre-Lecourbe/Volontaires. The apartment offers quality services and has been completely renovated. Calm and clear, it is located on the first floor courtyard and well isolated from street noise. It features a fully equipped kitchen with American bar, a lounge with a convertible sofa bed, a bathroom with tub and toilet, and finally the two bedrooms each with a double bed  and many closets.<br /><br />You will appreciate and enjoy your stay in Paris in our location w

In [9]:
descriptions = listings_d[["description"]]
descriptions = descriptions[descriptions["description"].notnull()]
descriptions["description"] = descriptions["description"].apply(str)

In [10]:
descriptions["description"] = descriptions["description"].apply(str.lower)

In [11]:
#descriptions.to_pickle("./descriptions_fr.pkl")

### Checkpoint

In [12]:
descriptions = pd.read_pickle("./descriptions_fr.pkl")

In [13]:
descriptions.head()

,description,langue
0,"100 m2 loft (1100 sq feet) with high ceiling, ...",False
1,<b>the space</b><br />i bedroom appartment in ...,False
2,"cozy, well-appointed and graciously designed s...",False
3,"very convenient, with the best location !<br /...",False
4,very large & nice apartment all for you! <br /...,False


In [14]:
def norm_tags(chaine):
    chaine = chaine + '</b>' #<--- ajout de balise car les descriptions sont tranquées (existence de la balise de début <b> mais absence de </b>)
    space = re.compile('<b>t.*?</b>')
    access = re.compile('<b>g.*?</b>')
    misc = re.compile('<b>o.*?</b>')
    license = re.compile('<b>l.*?</b>')
    transfo_list=[space, access, misc, license]
    replaced = ["<b>space</b>",'<b>access</b>','<b>misc</b>','<b>licence</b>']
    for i in range(4):
        chaine = re.sub(transfo_list[i], replaced[i], chaine)
    return chaine

In [15]:
def remove_noise(liste):
    try:
        liste.remove('<b></b>')
    except:
        pass
    return liste

In [16]:
soup = BeautifulSoup(descriptions["description"][2100])
[str(elt) for elt in soup.findAll("b")]

['<b>the space</b>',
 '<b>guest access</b>',
 '<b>other things to note</b>',
 '<b>license number</b>']

In [17]:
descriptions['BfSoup'] = descriptions["description"].apply(lambda x: [str(elt) for elt in BeautifulSoup(x).findAll("b")])

/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:314: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [18]:
descriptions.head()

,description,langue,BfSoup
0,"100 m2 loft (1100 sq feet) with high ceiling, ...",False,[<b>the space</b>]
1,<b>the space</b><br />i bedroom appartment in ...,False,[<b>the space</b>]
2,"cozy, well-appointed and graciously designed s...",False,"[<b>the space</b>, <b>guest access</b>]"
3,"very convenient, with the best location !<br /...",False,[<b>the space</b>]
4,very large & nice apartment all for you! <br /...,False,[<b>the space</b>]


In [19]:
descriptions["description"] = descriptions["description"].apply(norm_tags)

In [20]:
descriptions['BfSoup'] = descriptions["description"].apply(lambda x: [str(elt) for elt in BeautifulSoup(x).findAll("b")])

In [21]:
descriptions.head()

,description,langue,BfSoup
0,"100 m2 loft (1100 sq feet) with high ceiling, ...",False,[<b>space</b>]
1,<b>space</b><br />i bedroom appartment in pari...,False,[<b>space</b>]
2,"cozy, well-appointed and graciously designed s...",False,"[<b>space</b>, <b>access</b>]"
3,"very convenient, with the best location !<br /...",False,[<b>space</b>]
4,very large & nice apartment all for you! <br /...,False,[<b>space</b>]


In [22]:
descriptions['BfSoup'] = descriptions["BfSoup"].apply(lambda x: str(x))

In [23]:
descriptions

,description,langue,BfSoup
0,"100 m2 loft (1100 sq feet) with high ceiling, ...",False,['<b>space</b>']
1,<b>space</b><br />i bedroom appartment in pari...,False,['<b>space</b>']
2,"cozy, well-appointed and graciously designed s...",False,"['<b>space</b>', '<b>access</b>']"
3,"very convenient, with the best location !<br /...",False,['<b>space</b>']
4,very large & nice apartment all for you! <br /...,False,['<b>space</b>']
...,...,...,...
63085,studio entièrement refait à neuf idéalement si...,True,"['<b>space</b>', '<b>licence</b>']"
63086,"superbe appartement ambiance ""bohème-chic"" en ...",True,['<b>licence</b>']
63087,<b>licence</b><br />7510404886302</b>,False,['<b>licence</b>']
63088,charmant appartement au xvième arrondissement ...,True,"['<b>space</b>', '<b>access</b>']"


In [24]:
descriptions['BfSoup'] = descriptions["BfSoup"].apply(lambda x: remove_noise(x))

In [25]:
def order_tags(chaine, tag):
    tag_all = '<b>'+tag+'</b>'
    soup_chain = BeautifulSoup(chaine)
    soup = [str(elt) for elt in soup_chain.findAll("b")]
    if tag_all in chaine:
        if(soup == [] and tag == 'space'):
            return chaine
    
        chaine = chaine.split(tag_all)
        chaine[1] = chaine[1].split("<b>")[0]
        if(tag != 'space'):
            return chaine[1]
        if(tag == 'space'):
            return chaine[0]+' '+chaine[1] 
    else:
        return ''

In [26]:
order_tags(descriptions["description"][2], 'access')

'<br />the flat includes :<br /> - a redone bathroom <br />- a kitchenette fully equipped (oven, fridge, hob, kitchen utensil, kettle, washing machine...)<br />- a desk<br />- a comfortable sofa-bed<br />- a storage for your stuffs <br />- wi-fi<br /><br />towels, hairdryer a</b>'

In [27]:
descriptions["space"] = descriptions["description"].apply(lambda x: order_tags(x,'space'))

In [28]:
descriptions.head()

,description,langue,BfSoup,space
0,"100 m2 loft (1100 sq feet) with high ceiling, ...",False,['<b>space</b>'],"100 m2 loft (1100 sq feet) with high ceiling, ..."
1,<b>space</b><br />i bedroom appartment in pari...,False,['<b>space</b>'],<br />i bedroom appartment in paris 14</b>
2,"cozy, well-appointed and graciously designed s...",False,"['<b>space</b>', '<b>access</b>']","cozy, well-appointed and graciously designed s..."
3,"very convenient, with the best location !<br /...",False,['<b>space</b>'],"very convenient, with the best location !<br /..."
4,very large & nice apartment all for you! <br /...,False,['<b>space</b>'],very large & nice apartment all for you! <br /...


In [29]:
def del_tags(chaine):
    p = re.compile("<.*?>")
    return re.sub(p,'',chaine)

In [30]:
descriptions["space"] = descriptions["description"].apply(lambda x: del_tags(x))

In [31]:
descriptions.head()

,description,langue,BfSoup,space
0,"100 m2 loft (1100 sq feet) with high ceiling, ...",False,['<b>space</b>'],"100 m2 loft (1100 sq feet) with high ceiling, ..."
1,<b>space</b><br />i bedroom appartment in pari...,False,['<b>space</b>'],spacei bedroom appartment in paris 14
2,"cozy, well-appointed and graciously designed s...",False,"['<b>space</b>', '<b>access</b>']","cozy, well-appointed and graciously designed s..."
3,"very convenient, with the best location !<br /...",False,['<b>space</b>'],"very convenient, with the best location !space..."
4,very large & nice apartment all for you! <br /...,False,['<b>space</b>'],very large & nice apartment all for you! - sun...


In [32]:
nlp = spacy.load("fr_core_news_sm")

In [33]:
def keep_fr(chaine):
    doc = nlp(chaine)
    sentences = [str(elt) for elt in doc.sents]
    chaine_fr = ''
    for i in range(len(sentences)):
        if(cld3.get_language(sentences[i])[0] == 'fr'):
            chaine_fr = chaine_fr+sentences[i]
    return chaine_fr

In [34]:
descriptions["space"][0]

'100 m2 loft (1100 sq feet) with high ceiling, open modern kitchen, cable tv, wifi, sleeping space up to 5 people, republique area, 5 minutes from the st martin canal, marais area, 10 min from chatelet and the louvre.spacethe district has any service or shop you may desire at a walking distance, many restaurants and clubs as well. i will be pleased to suggest our best addresses and give you directions to have a very enjoyable journey. we have 1 separated bedroom and one mezzanine for the second bed. one bathroom is "en suite" to the main bedroom. welcome to our beautiful loft!karine'

In [35]:
keep_fr(descriptions["space"][0])

''

In [36]:
descriptions.shape

(61681, 4)

In [37]:
descriptions["space"] = descriptions["space"].apply(lambda x: keep_fr(x))

In [38]:
descriptions = descriptions[descriptions.space != '']

In [39]:
descriptions.shape

(35296, 4)

In [41]:
descriptions

,description,langue,BfSoup,space
6,"je suis une dame retraitée, qui propose un agr...",True,['<b>space</b>'],"je suis une dame retraitée, qui propose un agr..."
10,numéro d'enregistrement ville de paris 7511100...,False,['<b>space</b>'],numéro d'enregistrementville de paris751110053...
12,mon appartement est situé dans une très jolie ...,True,"['<b>space</b>', '<b>access</b>', '<b>misc</b>...",mon appartement est situé dans une très jolie ...
14,"<b>space</b><br />chambre très claire, calme, ...",True,['<b>space</b>'],"spacechambre très claire, calme, sur le jardin..."
22,loft across from the parc de la vilette<br />3...,False,"['<b>space</b>', '<b>access</b>', '<b>licence<...",loft across from the parc de la vilette3
...,...,...,...,...
63081,modern & newly refurbished apartment of 50sqm ...,False,"['<b>space</b>', '<b>access</b>', '<b>misc</b>']","is at 5-7 rue du faubourg poissonnière, 75009...."
63085,studio entièrement refait à neuf idéalement si...,True,"['<b>space</b>', '<b>licence</b>']",studio entièrement refait à neuf idéalement si...
63086,"superbe appartement ambiance ""bohème-chic"" en ...",True,['<b>licence</b>'],"appartement confortable, idéal pour passer que..."
63088,charmant appartement au xvième arrondissement ...,True,"['<b>space</b>', '<b>access</b>']",charmant appartement au xvième arrondissement ...


In [40]:
descriptions.to_pickle("./descriptions_clean_fr2.pkl")

In [3]:
descriptions = pd.read_pickle("./descriptions_clean_fr2.pkl")

### Modèle LSTM

In [4]:
corpus = list(descriptions.space)

In [5]:
len(corpus)

35296

In [6]:
corpus = list(descriptions.space)[:1000]

In [7]:
tokenizer = Tokenizer()

In [8]:
def gen_seq_from_tokens(corpus):
    tokenizer.fit_on_texts(corpus)
    max_words = len(tokenizer.word_index) + 1
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, max_words

In [9]:
inp_sequences, total_words = gen_seq_from_tokens(corpus)

In [10]:
len(inp_sequences)

61442

In [11]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len


In [12]:
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [13]:
total_words

6273

In [14]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(64))
    #model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

In [15]:
model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 173, 10)           62730     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                19200     
_________________________________________________________________
dense (Dense)                (None, 6273)              407745    
Total params: 489,675
Trainable params: 489,675
Non-trainable params: 0
_________________________________________________________________


In [16]:
early_stopping = EarlyStopping()

In [17]:
#model.fit(predictors, label, batch_size=0, epochs=100, verbose=0, callbacks=[early_stopping])

In [18]:
#model.save_weights('./checkpoints_dir/checkpoints/pop_1k_epoch_100')

In [23]:
#model.load_weights('./checkpoints/pop_1k_epoch_25')
model.load_weights('./checkpoints_dir/checkpoints/pop_1k_epoch_100')


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.embeddings.Embedding object at 0x7f2d90635c90> and <tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x7f2d914bb350>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x7f2d914bb350> and <tensorflow.python.keras.layers.core.Dense object at 0x7f2d8e201790>).


In [27]:
#tf.keras.models.save_model(model, "my_model.hp5", save_format="h5")

In [29]:
#model1 = tf.keras.models.load_model("my_model.hp5")

AttributeError: 'str' object has no attribute 'decode'

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 173, 10)           62730     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                19200     
_________________________________________________________________
dense (Dense)                (None, 6273)              407745    
Total params: 489,675
Trainable params: 489,675
Non-trainable params: 0
_________________________________________________________________


In [17]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [2]:
generate_text("chambre", 30,model, max_sequence_len)

NameError: name 'model' is not defined

### Modèle 2 25 epochs

In [73]:
generate_text("chambre", 30,model, max_sequence_len)

'Chambre De Paris République De Paris Min De La République Et Un Une Salle De La Chambre De La Chambre De La Et Et Un Salle De La Chambre De La'

In [33]:
generate_text("chambre", 30,model1, max_sequence_len)

'Chambre De 2 Pièces Au 6Ème Étage Avec Ascenseur Et Très Calme Et Très Lumineux Et Très Calme Et Très Lumineux Et Calme Il Est Très Lumineux Et Calme Il Est'

In [79]:
generate_text("mont", 10,model, max_sequence_len)

'Mont Paris Paris Place De Paris Saint Germain De Paris Saint'

In [77]:
generate_text("appartement", 10,model, max_sequence_len)

'Appartement The Halles Of Paris Paris The Heart Of Paris Metro'

### Modèle 2 100 epochs

In [19]:
generate_text("le", 30,model, max_sequence_len)

"Le Studio Est Situé Au Cœur De Paris Des Années Magasins A Pied De De Maison Pour Un Marché L'Emplacement Chateaubriand Le Seine Et Des Champs Elysée Ou 2 Pièces 50"

In [20]:
generate_text("appartement", 30,model, max_sequence_len)

'Appartement De 40M2 Avec Pierres Apparentes Peut Accueillir Une Petite Salle De Bain Avec Baignoire Et Ensoleillé Au Premier Étage Avec Ascenseur Vue Sur La Rue De Bretagne Extrêmement Calme Et'

In [22]:
generate_text("paris", 30,model, max_sequence_len)

"Paris 11Th Arrondissement Rue De L'Orillon Metro Station Goncourt Or Parmentier Or Republique Ligne 2 Stations Rer 9 Metro Station 5 Station 5 Metro Station Ligne 2 5 Metro Stations Ligne"

In [23]:
generate_text("appartement très calme", 30,model, max_sequence_len)

'Appartement Très Calme Et Ensoleillé De 100 M2 Situé Dans Un Quartier Calme Et Animé Accesstout 4 Pièces De 45 M² Et Un Grand Studio Ouvre Sur Le Jardin Et Les Cours Fenêtres'